In [1]:
!pip install web3

In [2]:
import time
import json
import requests
import pandas as pd
from tqdm import tqdm
from web3 import Web3
from web3.exceptions import ABIFunctionNotFound

In [3]:
BSC_API_KEY = "V1A3UZAFFKSNQUHCWA6MYDZ31364JDBGSJ"
BSC_URL = 'https://api.bscscan.com/api'

web3ProviderURL = "https://rpc.ankr.com/bsc/b1bdd2819d3da7be4760d270af7ef985a74725cbe337c348bca633d096806c5a"
w3 = Web3(Web3.HTTPProvider(web3ProviderURL))

getAccountBalance_url = "https://rpc.ankr.com/multichain/79258ce7f7ee046decc3b5292a24eb4bf7c910d7e39b691384c7ce0cfb839a01/?ankr_getAccountBalance="

In [4]:
topics = [
    "0x783cca1c0412dd0d695e784568c96da2e9c22ff989357a2e8b1d9b2b4e6b7118"
]

In [5]:
factories = {
    "pancakeswap_v3": {
        "address" : ["0x0bfbcf9fa4f9c56b0f40a671ad40e0805a091865"],
        "blockInterval" : 150000,
        },
}

In [6]:
# Get the number of most recent block
def get_latest_block():
    params = {
        "module": "proxy",
        "action": "eth_blockNumber",
        "apikey": BSC_API_KEY
    }
    response = requests.post(BSC_URL, params=params)
    data = json.loads(response.text)
    latest_block = int(data['result'], 16)
    return latest_block

In [7]:
def get_abi(address):
    params = {
        "module": "contract",
        "action": "getabi",
        "address": address,
        "apikey": BSC_API_KEY
    }
    response = requests.post(BSC_URL, params=params)
    time.sleep(0.2)
    data = json.loads(response.text)
    result = data['result']
    return result

In [8]:
abi_templete = get_abi("0xa7619d726f619062d2d2bcadbb2ee1fb1952d6d7")

def get_contract(address):
    try:
        abi = get_abi(address)
        checksum_address = Web3.to_checksum_address(address)
        contract = w3.eth.contract(address=checksum_address, abi=abi)
        return contract
    except Exception:
        checksum_address = Web3.to_checksum_address(address)
        contract = w3.eth.contract(address=checksum_address, abi=abi_templete)
        return contract

In [9]:
def get_fee_from_fee(contract, name):
    deno_dict = {
        "pancakeswap_v3": 10000*100,
        "uniswap_v3": 10000*100,
        "vyper_contract": 10**10,
    }
    if name in deno_dict:
        fee = float(contract.functions.fee().call())
        fee = fee / deno_dict[name]
        return fee
    elif name == "fraxswap":
        fee = 10000 - float(contract.functions.fee().call())
        fee = fee / 10000
        return fee
    return None


# Try get fee from swapFee function in abi (Biswap, Nomiswap, Global LPs, Fins)
def get_fee_from_swapFee(contract, name):
    deno_dict = {
        "biswap": 1000,
        "nomiswap": 1000,
        "nomiswapStable": 100000,
        "global": 100000,
        "fins": 10000,
        "plearn": 1000
    }

    if name in deno_dict:
        swap_fee = float(contract.functions.swapFee().call())
        fee = swap_fee / deno_dict[name]
        return fee
    return None


def get_fee_for_thena(contract, name):
    try:
        isStable = bool(contract.functions.stable().call())
        if isStable:
            fee = 4 / (100*100)
        else:
            fee = 18 / (100*100)
        return fee
    except Exception as e:
        return None


def get_fee(contract, name):
    fix_fee_dict = {
        "apeswap" : 2/1000,
        "pancake": 25/10000,
        "uniswap_v2": 3/1000,
        "sushiswap" : 3/1000,
        "babydoge": 100/1000,
    }

    fee_fun = [
        get_fee_from_fee,
        get_fee_from_swapFee,
        get_fee_for_thena
    ]

    if name in fix_fee_dict:
        return fix_fee_dict[name]

    for fun in fee_fun:
        fee = fun(contract, name)
        if fee:
            return fee
    return fee

In [10]:
def get_tokens(contract):
    try:
        token0 = contract.functions.token0().call().lower()
        token1 = contract.functions.token1().call().lower()
        return token0, token1
    except Exception:
        return None, None

In [11]:
def get_tvl(contract):
    address = contract.address
    
    token_type = ['NATIVE']
    payload = {
        "jsonrpc": "2.0",
        "method": "ankr_getAccountBalance",
        "params": {
            "blockchain": ["bsc"],
            "walletAddress": address,
            "nativeFirst": True,
            "onlyWhitelisted": True,
            "pageSize": 2
        },
        "id": 1
    }
    headers = {
        "accept": "application/json",
        "content-type": "application/json"
    }

    response = requests.post(getAccountBalance_url, json=payload, headers=headers)
    time.sleep(1)
    if response.status_code == 200:
        data = json.loads(response.text)
        tvl = data['result']['totalBalanceUsd']
        return tvl
    return None

In [12]:
def get_pool_from_events(factory, start_block, end_block):
    pool = []
    name = factory
    for factory_address in factories[factory]["address"]:
        for topic in topics:
            params = {
                "module": "logs",
                "action": "getLogs",
                "fromBlock": start_block,
                "toBlock": end_block,
                "address" : factory_address,
                "topic0": topic,
                "apikey": BSC_API_KEY
            }
            response = requests.post(BSC_URL, params=params)
            time.sleep(0.2)
            data = json.loads(response.text)
            logs = data['result']
            for i in tqdm(range(len(logs))):
                try:
                    log = logs[i]
                    hex_string = log['data']
                    hex_string = hex_string[2:]
                    address = '0x' + hex_string[-40:].lower()
                    contract = get_contract(address)

                    tvl = get_tvl(contract)
                    fee = get_fee(contract, name)
                    token0, token1 = get_tokens(contract)
                    row = {
                        "name" : name,
                        "factory" : factory_address,
                        "tvl" : tvl,
                        "fee" : fee,
                        "contract" : address,
                        "token0" : token0,
                        "token1" : token1,
                    }
                    pool.append(row)
                except Exception:
                    continue
    return pool

In [13]:
def to_file(pool, count):
    pool_df = pd.DataFrame(pool)
    pool_df = pool_df.dropna(axis=0).reset_index(drop=True)
    
    file_name = f"part{count}.csv"
    pool_df.to_csv(file_name, index=False)
    return file_name

In [14]:
latest_block = get_latest_block()
latest_block

33668205

In [15]:
files = []
for factory in factories.keys():
    interval = factories[factory]["blockInterval"]
    for count in range(0,1000):
        start = count * interval
        end = start + interval
        if end > latest_block:
            end = latest_block
        pool = get_pool_from_events(factory, start, end)
        if len(pool) > 0:
            file_name = to_file(pool, count)
            files.append(file_name)
        if end >= latest_block:
            break

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, 

KeyboardInterrupt: 

In [ ]:
combined_df = pd.DataFrame()
for file in files:
    df = pd.read_csv(file)
    combined_df = pd.concat([combined_df, df], ignore_index=True)
    
combined_df = combined_df.drop_duplicates()
combined_df

In [ ]:
combined_df.to_csv('bsc_pancakeswapv3.csv', index=False)